# Tensor Flow Neural Network utilizing Tweet Data as well as Stock/SP500 data
This model, from my testing, could not surpass the Random Forest Model as being more accurate, and due lack of twitter data, this model was never used in the final program.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from sklearn.preprocessing import MinMaxScaler

data = pd.read_excel("Data_all082323.xlsx")
data_clean = data.drop(["date","sp_rf"], axis=1)
print(data_clean.info())

# Check Balance
print(data_clean['Predict'].value_counts())

In [ ]:
batch_size = 250
epoch_no = 100
lr = .0001

train_data, test_data = train_test_split(data_clean, test_size=0.2)
scaler = MinMaxScaler()


X_train = train_data.drop('Predict', axis=1)
X_train_col = X_train.columns
X_train = X_train.astype('float32')
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns=X_train_col)
y_train = train_data['Predict']


X_test = test_data.drop('Predict', axis=1)
X_test_col = X_test.columns
X_test = X_test.astype('float32')
X_test = scaler.fit_transform(X_test)
X_test = pd.DataFrame(X_test, columns=X_test_col)
y_test = test_data['Predict']



train_dataset = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values)).shuffle(len(train_data)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values)).batch(batch_size)

In [ ]:
# Building
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(16,)),
    layers.Dense(32, activation='relu'),# Add more layers here
    layers.Dense(1, activation='sigmoid')  # sigmoid for binary classification
])

In [ ]:
# Compling
adam_optimizer = optimizers.Adam(learning_rate=lr)
#adam_optimizer = 'adam'

model.compile(optimizer=adam_optimizer,
              loss='binary_crossentropy',  # appropriate for binary classification
              metrics=['accuracy'])

In [ ]:
# Training

history = model.fit(train_dataset, epochs=epoch_no, validation_data=test_dataset)

In [ ]:
# Determining Accruacy

# Extract accuracy values
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Create epochs range
epochs = range(1, len(train_acc) + 1)

# Plot training and validation accuracy
plt.figure(figsize=(12, 6))
plt.plot(epochs, train_acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

## Saving Model

In [ ]:
model_path = 'NN_stockuniq_model'
model.save(model_path)